```plaintext
Procedure QueryDataFromDBMS:
    Define A as:
        Select 
            fields: COM_ID, CSL_ID, CNTR_ID, COST_C, COST_DESC, COST_RATE, COST_UNIT, COST_QTY, 
                    COST_TERM with NULL replaced by 'FP', PARTY_ID, COST_CURR, COST_EX_RATE,
                    LOC_AMT, DOC_AMT, DOC_TYPE, DOC_NO, POL_ID, POD_ID, POD, VESSEL, ETD_POL_D,
                    ETA_POD_D, CNTR_SIZE, CNTR_TYPE, PARTY_NAME, ETA_ETD_NO
        From 
            Tables: COSTING (aliased as CT), CONSOL (aliased as C), CONSOL_CNTR (aliased as CC), TRADER (aliased as T)
        Where 
            conditions: C.COM_ID is '010', C.JOB_TYPE is 'CS', CT.COM_ID is equal to C.COM_ID,
                        CT.CSL_ID is equal to C.CSL_ID, C.COST_CENTER is 'E', CT.COM_ID is equal to CC.COM_ID,
                        CT.CNTR_ID is equal to CC.CNTR_ID, CT.PARTY_ID is equal to T.PARTY_ID,
                        CT.COST_C is 'OF-E-CT-CR', CT.COST_CURR is 'USD', CT.BKG_NO is NULL

    Query:
        Select 
            fields: COM_ID, Maximum of PARTY_ID, Maximum of PARTY_NAME, CSL_ID, ETA_ETD_NO,
                    Maximum of POD_ID, Maximum of POD, Maximum of ETD_POL_D, CNTR_ID, Maximum of CNTR_SIZE,
                    Maximum of CNTR_TYPE, COST_TERM, Absolute sum of COST_RATE
        From
            Table: A
        Group By 
            fields: COM_ID, CSL_ID, ETA_ETD_NO, CNTR_ID, COST_TERM


```plaintext
Procedure CreateOrReplaceView:
    Create or replace force editionable view "VW_CR_COST_OAC" with fields: 
        "COM_ID", "PARTY_ID", "PARTY_NAME", "CSL_ID", "ETA_ETD_NO", "ETD_POL_D", 
        "POD_ID", "POD", "CNTR_ID", "CNTR_SIZE", "CNTR_TYPE", "COST_TERM", "RATE", 
        "ENCODED_TYPE", "ENCODED_TERM", "POD_ID_EN", default collation "USING_NLS_COMP"
    
    Define view with:
        Select 
            fields: "COM_ID", "PARTY_ID", "PARTY_NAME", "CSL_ID", "ETA_ETD_NO", "ETD_POL_D",
                    "POD_ID", "POD", "CNTR_ID", "CNTR_SIZE", "CNTR_TYPE", "COST_TERM", "RATE",
                    Case: 
                        When CNTR_TYPE is 'HC' Then 1
                        When CNTR_TYPE is 'GP' Then 2
                        When CNTR_TYPE is 'GP (SOC)' Then 3
                        When CNTR_TYPE is 'HC NOR' Then 4
                        When CNTR_TYPE is 'HC (SOC)' Then 5
                        When CNTR_TYPE is 'HC (DG)' Then 6
                    End as "ENCODED_TYPE",
                    Case:
                        When COST_TERM is 'FC' Then 1
                        When COST_TERM is 'FP' Then 2
                    End as "ENCODED_TERM",
                    ORA_HASH(POD_ID) as "POD_ID_EN"
        From
            Table: OAC_CR_COST